In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ATLAS_style import *
ROOT.gErrorIgnoreLevel = ROOT.kError
set_atlas_style()

In [2]:
file_HIST_z = ROOT.TFile.Open("jan25/root_files/z/HIST.root")
file_HIST_gamma = ROOT.TFile.Open("jan25/root_files/gamma/HIST.root")
#file_HIST_tt = ROOT.TFile.Open("jan25/root_files/tt/HIST.root")
file_HIST_eb = ROOT.TFile.Open("jan25/root_files/eb/HIST.root")

file_monitoring_z = ROOT.TFile.Open("jan25/root_files/z/expert-monitoring.root")
file_monitoring_gamma = ROOT.TFile.Open("jan25/root_files/gamma/expert-monitoring.root")
#file_monitoring_tt = ROOT.TFile.Open("jan25/root_files/tt/expert-monitoring.root")
file_monitoring_eb = ROOT.TFile.Open("jan25/root_files/eb/HIST_HLTMON.root")

## directories

In [ ]:
def get_HLT_dir(HIST_file, base_dir, keyword=None):
    file_base_dir = HIST_file.Get(base_dir)
    HLT_dir = []

    keys = file_base_dir.GetListOfKeys()
    for key in keys:
        obj = key.ReadObj()
        if obj.InheritsFrom("TDirectory"):
            name = obj.GetName()
            if keyword is not None:
                if keyword in name:
                    HLT_dir.append(name)
            else:
                HLT_dir.append(name)

    return HLT_dir

HLT_dir = get_HLT_dir(HIST_file = file_HIST_z,
                      base_dir = 'run_470000/HLT/TauMon/HLT_Efficiency',
                      #keyword = "GNTau"
                      )
print(HLT_dir)

Eff_dir = ['HLT_Efficiency_1P', 'HLT_Efficiency_3P']
Eff_plot = ['EffHLT_tauEta_highPt_wrt_Offline', 'EffHLT_tauEta_wrt_Offline',
            'EffHLT_tauPt_coarse_wrt_Offline', 'EffHLT_tauPt_wrt_Offline'
            ]

Var_dir = ['HLT_0P', 'HLT_1P', 'HLT_MP']
gnn_scores = ['GNTau_TauIDScore', 'GNTau_TauIDScoreSigTrans']
rnn_scores = ['DeepSet_TauIDScore', 'DeepSet_TauIDScoreSigTrans']

## efficiencies

In [6]:
def compare_eff(path, eff_1, eff_2, label_1, label_2, out_dir ,ymin=0, ymax=1.6):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    eff_1.SetLineColor(ROOT.kBlack)
    eff_2.SetLineColor(ROOT.kRed)
    
    eff_1.Draw("AP")
    eff_2.Draw("P SAME")

    canvas.Update()

    graph_1 = eff_1.GetPaintedGraph()
    if graph_1:
        graph_1.GetYaxis().SetRangeUser(ymin, ymax)

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/HLT_Efficiency/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_Efficiency_3P/EffHLT_tauPt_wrt_Offline'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '1P' in path_parts[5]:
        prong = '1-prong'
    elif '3P' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.19, 0.87, 0.45, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')' + ' ' + path_parts[6])
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(eff_1, label_1, "lp")
    legend.AddEntry(eff_2, label_2, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.2, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for eff_dir in Eff_dir:
        for eff_plot in Eff_plot:
            path = 'run_470000/HLT/TauMon/HLT_Efficiency' + '/' + hlt_dir + '/' + eff_dir + '/' + eff_plot
            compare_eff(path=path,
                        eff_1=file_HIST_z.Get(path),
                        eff_2=file_HIST_gamma.Get(path),
                        label_1='Ztautau',
                        label_2='Gammatautau',
                        out_dir='jan25/efficiencies/compare/',
                        ymin=0,
                        ymax=1.6
                       )

## scores

In [ ]:
def compare_var(path, var_1, var_2, label_1, label_2, out_dir):    
    canvas = ROOT.TCanvas(path, "", 1000, 800)
    
    var_1.Scale(1.0 / var_1.Integral())
    var_2.Scale(1.0 / var_2.Integral())

    max_1 = var_1.GetBinContent(var_1.GetMaximumBin())
    max_2 = var_2.GetBinContent(var_2.GetMaximumBin())
    var_1.SetMaximum(np.max((max_1, max_2)) * 20)
    if "SigTrans" in path:
        var_1.GetXaxis().SetRangeUser(0, 1.05)
        var_2.GetXaxis().SetRangeUser(0, 1.05)
    else:
        var_1.GetXaxis().SetRangeUser(0, 8)
        var_2.GetXaxis().SetRangeUser(0, 8)

    y_axis = var_1.GetYaxis()
    y_axis.SetTitle("A.U. (norm = 1)")
    y_axis.SetTitleOffset(2.2)

    var_2.SetLineColor(ROOT.kBlack)
    var_2.SetMarkerColor(ROOT.kBlack)
    var_1.SetLineColor(ROOT.kRed)
    var_1.SetMarkerColor(ROOT.kRed)
    
    var_1.Draw("HIST")
    var_2.Draw("HIST,same")

    canvas.SetLogy(1)

    canvas.Update()

    canvas.Modified()
    canvas.Update()

    # 'run_470000/HLT/TauMon/basicVars/HLT_tau80_mediumRNN_tracktwoMVA_L1eTAU80/HLT_3P/Pt'
    
    path_parts = path.split('/')
    prong = 'NA'
    if '0P' in path_parts[5]:
        prong = '0-prong'
    elif '1P' in path_parts[5]:
        prong = '1-prong'
    elif 'MP' in path_parts[5]:
        prong = '3-prong'
    
    legend = ROOT.TLegend(0.49, 0.87, 0.75, 0.7)
    legend.SetHeader(path_parts[4] + ' ' + '(' + prong + ')')
    
    header_entry = legend.GetListOfPrimitives().At(0)
    header_entry.SetTextSize(17)
    legend.SetTextSize(17)
    legend.AddEntry(var_1, label_1, "lp")
    legend.AddEntry(var_2, label_2, "lp")
    legend.Draw()

    canvas.Update()
    ATLASlabel(0.5, 0.88, "Internal")

    save_dir = out_dir + path_parts[4] + '/' + prong
    if not os.path.exists(save_dir):
        !mkdir -p {save_dir}
    canvas.SaveAs(save_dir + '/' + path_parts[6] + '.png')

    del canvas, legend

for hlt_dir in HLT_dir:
    for var_dir in Var_dir:
        for score in gnn_scores:
            path = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + score
            try:
                compare_var(path=path,
                            var_2=file_HIST_z.Get(path),
                            var_1=file_HIST_gamma.Get(path),
                            label_2='Ztautau (GNTau)',
                            label_1='Gammatautau (GNTau)',
                            out_dir='jan25/scores/compare_samples/'
                        )
            except:
                pass
'''
for hlt_dir in HLT_dir:
    for var_dir in Var_dir:
        for score1, score2 in zip(gnn_scores, rnn_scores):
            path1 = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + score1
            path2 = 'run_470000/HLT/TauMon/basicVars' + '/' + hlt_dir + '/' + var_dir + '/' + score2
            try:
                compare_var(path=path1,
                            var_1=file_HIST_z.Get(path1),
                            var_2=file_HIST_z.Get(path2),
                            label_2='Ztautau (DeepSet)',
                            label_1='Ztautau (GNTau)',
                            out_dir='jan25/scores/compare_scores/z/'
                        )
            except:
                pass

            try:
                compare_var(path=path1,
                            var_1=file_HIST_gamma.Get(path1),
                            var_2=file_HIST_gamma.Get(path2),
                            label_2='Gammatautau (DeepSet)',
                            label_1='Gammatautau (GNTau)',
                            out_dir='jan25/scores/compare_scores/gamma/'
                        )
            except:
                pass
'''